# **AIHT FINAL PROJECT**

# Code for Data Modelling

In [217]:
library(dplyr)
library(lubridate)
library(readr)

In [218]:
# Install the arrow package
if (!require("arrow")) install.packages("arrow")

In [219]:
# Load the arrow package
library(arrow)

# Read the Parquet file
df <- read_parquet("taxidata.parquet")


# Display the first few rows of the data
head(df)

#this table(which is a flat hierarchy) needs to be converted into fact and dimension table format (dimensional modeling). Will be done using lucid chart.


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
<int>,<dttm>,<dttm>,<int>,<dbl>,<int>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2025-01-01 00:18:38,2025-01-01 00:26:59,1,1.60,1,N,229,237,1,10.0,3.5,0.5,3.00,0,1,18.00,2.5,0
1,2025-01-01 00:32:40,2025-01-01 00:35:13,1,0.50,1,N,236,237,1,5.1,3.5,0.5,2.02,0,1,12.12,2.5,0
1,2025-01-01 00:44:04,2025-01-01 00:46:01,1,0.60,1,N,141,141,1,5.1,3.5,0.5,2.00,0,1,12.10,2.5,0
2,2025-01-01 00:14:27,2025-01-01 00:20:01,3,0.52,1,N,244,244,2,7.2,1.0,0.5,0.00,0,1,9.70,0.0,0
2,2025-01-01 00:21:34,2025-01-01 00:25:06,3,0.66,1,N,244,116,2,5.8,1.0,0.5,0.00,0,1,8.30,0.0,0
2,2025-01-01 00:48:24,2025-01-01 01:08:26,2,2.63,1,N,239,68,2,19.1,1.0,0.5,0.00,0,1,24.10,2.5,0


In [220]:
glimpse(df)

Rows: 3,475,226
Columns: 19
$ VendorID              <int> 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, …
$ tpep_pickup_datetime  <dttm> 2025-01-01 00:18:38, 2025-01-01 00:32:40, 2025-…
$ tpep_dropoff_datetime <dttm> 2025-01-01 00:26:59, 2025-01-01 00:35:13, 2025-…
$ passenger_count       <int> 1, 1, 1, 3, 3, 2, 0, 0, 0, 1, 1, 1, 3, 1, 1, 3, …
$ trip_distance         <dbl> 1.60, 0.50, 0.60, 0.52, 0.66, 2.63, 0.40, 1.60, …
$ RatecodeID            <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, …
$ store_and_fwd_flag    <chr> "N", "N", "N", "N", "N", "N", "N", "N", "N", "N"…
$ PULocationID          <int> 229, 236, 141, 244, 244, 239, 170, 234, 148, 237…
$ DOLocationID          <int> 237, 237, 141, 244, 116, 68, 170, 148, 170, 262,…
$ payment_type          <int> 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 2, …
$ fare_amount           <dbl> 10.0, 5.1, 5.1, 7.2, 5.8, 19.1, 4.4, 12.1, 19.1,…
$ extra                 <dbl> 3.5, 3.5, 3.5, 1.0, 1.0, 1.0, 3.5, 3.5, 3.5, 1.0…
$ mta_tax   

In [221]:
# Convert to datetime
df$tpep_pickup_datetime <- ymd_hms(df$tpep_pickup_datetime)
df$tpep_dropoff_datetime <- ymd_hms(df$tpep_dropoff_datetime)

Warning message:
“ 70 failed to parse.”
Warning message:
“ 310 failed to parse.”


In [222]:
glimpse(df)

Rows: 3,475,226
Columns: 19
$ VendorID              <int> 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, …
$ tpep_pickup_datetime  <dttm> 2025-01-01 00:18:38, 2025-01-01 00:32:40, 2025-…
$ tpep_dropoff_datetime <dttm> 2025-01-01 00:26:59, 2025-01-01 00:35:13, 2025-…
$ passenger_count       <int> 1, 1, 1, 3, 3, 2, 0, 0, 0, 1, 1, 1, 3, 1, 1, 3, …
$ trip_distance         <dbl> 1.60, 0.50, 0.60, 0.52, 0.66, 2.63, 0.40, 1.60, …
$ RatecodeID            <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, …
$ store_and_fwd_flag    <chr> "N", "N", "N", "N", "N", "N", "N", "N", "N", "N"…
$ PULocationID          <int> 229, 236, 141, 244, 244, 239, 170, 234, 148, 237…
$ DOLocationID          <int> 237, 237, 141, 244, 116, 68, 170, 148, 170, 262,…
$ payment_type          <int> 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 2, …
$ fare_amount           <dbl> 10.0, 5.1, 5.1, 7.2, 5.8, 19.1, 4.4, 12.1, 19.1,…
$ extra                 <dbl> 3.5, 3.5, 3.5, 1.0, 1.0, 1.0, 3.5, 3.5, 3.5, 1.0…
$ mta_tax   

In [223]:
# Create datetime dimension
datetime_dim <- df %>%
  select(tpep_pickup_datetime, tpep_dropoff_datetime) %>%
  distinct() %>%
  mutate(
    pick_hour = hour(tpep_pickup_datetime),
    pick_day = day(tpep_pickup_datetime),
    pick_month = month(tpep_pickup_datetime),
    pick_year = year(tpep_pickup_datetime),
    pick_weekday = wday(tpep_pickup_datetime, label = FALSE) - 1,
    drop_hour = hour(tpep_dropoff_datetime),
    drop_day = day(tpep_dropoff_datetime),
    drop_month = month(tpep_dropoff_datetime),
    drop_year = year(tpep_dropoff_datetime),
    drop_weekday = wday(tpep_dropoff_datetime, label = FALSE) - 1,
    datetime_id = row_number()
  ) %>%
  select(datetime_id, tpep_pickup_datetime, pick_hour, pick_day, pick_month, pick_year, pick_weekday,
         tpep_dropoff_datetime, drop_hour, drop_day, drop_month, drop_year, drop_weekday)

In [224]:
datetime_dim

datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
<int>,<dttm>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dttm>,<int>,<int>,<dbl>,<dbl>,<dbl>
1,2025-01-01 00:18:38,0,1,1,2025,3,2025-01-01 00:26:59,0,1,1,2025,3
2,2025-01-01 00:32:40,0,1,1,2025,3,2025-01-01 00:35:13,0,1,1,2025,3
3,2025-01-01 00:44:04,0,1,1,2025,3,2025-01-01 00:46:01,0,1,1,2025,3
4,2025-01-01 00:14:27,0,1,1,2025,3,2025-01-01 00:20:01,0,1,1,2025,3
5,2025-01-01 00:21:34,0,1,1,2025,3,2025-01-01 00:25:06,0,1,1,2025,3
6,2025-01-01 00:48:24,0,1,1,2025,3,2025-01-01 01:08:26,1,1,1,2025,3
7,2025-01-01 00:14:47,0,1,1,2025,3,2025-01-01 00:16:15,0,1,1,2025,3
8,2025-01-01 00:39:27,0,1,1,2025,3,2025-01-01 00:51:51,0,1,1,2025,3
9,2025-01-01 00:53:43,0,1,1,2025,3,2025-01-01 01:13:23,1,1,1,2025,3


In [225]:
# Passenger count dimension
passenger_count_dim <- df %>%
  select(passenger_count) %>%
  distinct() %>%
  mutate(passenger_count_id = row_number()) %>%
  select(passenger_count_id, passenger_count)

In [226]:
head(passenger_count_dim)

passenger_count_id,passenger_count
<int>,<int>
1,1
2,3
3,2
4,0
5,4
6,9


In [227]:
# Trip distance dimension
trip_distance_dim <- df %>%
  select(trip_distance) %>%
  distinct() %>%
  mutate(trip_distance_id = row_number()) %>%
  select(trip_distance_id, trip_distance)

In [228]:
head(trip_distance_dim)

trip_distance_id,trip_distance
<int>,<dbl>
1,1.60
2,0.50
3,0.60
4,0.52
5,0.66
6,2.63


In [229]:
# Rate code dimension
rate_code_type <- c(
  "1" = "Standard rate",
  "2" = "JFK",
  "3" = "Newark",
  "4" = "Nassau or Westchester",
  "5" = "Negotiated fare",
  "6" = "Group ride"
)

# Rate code dimension (fixing NA rowname issue)
rate_code_dim <- df %>%
  select(RatecodeID) %>%
  distinct() %>%
  mutate(
    rate_code_id = row_number(),
    rate_code_name = rate_code_type[as.character(RatecodeID)]
  ) %>%
  select(rate_code_id, RatecodeID, rate_code_name) %>%
  as.data.frame()

# View
head(rate_code_dim, 5)



,rate_code_id,RatecodeID,rate_code_name
,<int>,<int>,<chr>
1,1,1,Standard rate
2,2,5,Negotiated fare
3,3,99,NA
4,4,2,JFK
5,5,4,Nassau or Westchester


In [230]:
head(rate_code_dim)

,rate_code_id,RatecodeID,rate_code_name
,<int>,<int>,<chr>
1,1,1,Standard rate
2,2,5,Negotiated fare
3,3,99,NA
4,4,2,JFK
5,5,4,Nassau or Westchester
6,6,3,Newark


In [231]:
# Pickup location dimension
pickup_location_dim <- df %>%
  select(PULocationID) %>%
  distinct() %>%
  mutate(pickup_location_id = row_number()) %>%
  select(pickup_location_id, PULocationID)

In [232]:
head(pickup_location_dim)

pickup_location_id,PULocationID
<int>,<int>
1,229
2,236
3,141
4,244
5,239
6,170


In [233]:
# Dropoff location dimension
dropoff_location_dim <- df %>%
  select(DOLocationID) %>%
  distinct() %>%
  mutate(dropoff_location_id = row_number()) %>%
  select(dropoff_location_id, DOLocationID)

In [234]:
head(dropoff_location_dim)

dropoff_location_id,DOLocationID
<int>,<int>
1,237
2,141
3,244
4,116
5,68
6,170


In [236]:
# Payment type dimension
payment_type_name <- c(
  "1" = "Credit card",
  "2" = "Cash",
  "3" = "No charge",
  "4" = "Dispute",
  "5" = "Unknown",
  "6" = "Voided trip"
)

payment_type_dim <- df %>%
  select(payment_type) %>%
  distinct() %>%  # Remove rowid_to_column here
  mutate(
    payment_type_id = row_number(),
    payment_type_name = payment_type_name[as.character(payment_type)]
  ) %>%
  select(payment_type_id, payment_type, payment_type_name) %>%
  as.data.frame() # Convert to data frame

head(payment_type_dim)

,payment_type_id,payment_type,payment_type_name
,<int>,<int>,<chr>
1,1,1,Credit card
2,2,2,Cash
3,3,4,Dispute
4,4,3,No charge
5,5,5,Unknown
6,6,0,NA


In [242]:
# Fact table
fact_table <- df %>%
  left_join(passenger_count_dim, by = "passenger_count") %>%
  left_join(trip_distance_dim, by = "trip_distance") %>%
  left_join(rate_code_dim, by = "RatecodeID") %>%
  left_join(pickup_location_dim, by = "PULocationID") %>%
  left_join(dropoff_location_dim, by = "DOLocationID") %>%
  left_join(datetime_dim, by = c("tpep_pickup_datetime", "tpep_dropoff_datetime")) %>%
  left_join(payment_type_dim, by = "payment_type") %>%
  select(VendorID, datetime_id, passenger_count_id,
         trip_distance_id, rate_code_id, store_and_fwd_flag,
         pickup_location_id, dropoff_location_id,
         payment_type_id, fare_amount, extra, mta_tax,
         tip_amount, tolls_amount, improvement_surcharge, total_amount,congestion_surcharge, Airport_fee)

head(fact_table)

VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1,1,1,N,1,1,1,10.0,3.5,0.5,3.00,0,1,18.00,2.5,0
1,2,1,2,1,N,2,1,1,5.1,3.5,0.5,2.02,0,1,12.12,2.5,0
1,3,1,3,1,N,3,2,1,5.1,3.5,0.5,2.00,0,1,12.10,2.5,0
2,4,2,4,1,N,4,3,2,7.2,1.0,0.5,0.00,0,1,9.70,0.0,0
2,5,2,5,1,N,4,4,2,5.8,1.0,0.5,0.00,0,1,8.30,0.0,0
2,6,3,6,1,N,5,5,2,19.1,1.0,0.5,0.00,0,1,24.10,2.5,0
